In [1]:
from copy import deepcopy
import pandas as pd
from enbios.base.experiment import Experiment
import bw2data

from enbios.bw2.util import report
from enbios.models.experiment_models import ExperimentData

In [2]:
# get an overview of brightway projects and databases
report()

Project: default
[]
Project: ecoinvent_391
['biosphere3', 'ecoinvent_391_cutoff']


In [3]:
# select the brightway project and database (some ecoinvent database)
PROJECT_NAME = "ecoinvent_391"
DATABASE = "ecoinvent_391_cutoff"

bw2data.projects.set_current(PROJECT_NAME)
db = bw2data.Database(DATABASE)

# Simple example experiment
In this first simple example we calculate the impact of 4 wind turbine activities in spain, using 2 different methods.

In [4]:
wind_turbines_spain = db.search(
    "electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"}
)[:2]
wind_turbines_spain

Excluding 319 filtered results


['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, ES, None)]

In [8]:
# for the experiment we need to create a list of activities (or a dict, where the keys represent the aliases)
# We need to add the codes, otherwise the brightway search will not be not uniquely identify the activities
# adding name is just for convenience
experiment_activities = []

for activity in wind_turbines_spain[:2]:
    experiment_activities.append(
        {"id": {"name": activity["name"], "code": activity["code"]}}
    )

In [6]:
# we can modify the output of the activities, by default it is the reference product (1 of the activity unit)
experiment_activities[0]["output"] = ["kilowatt_hour", 3]
experiment_activities

[{'id': {'name': 'electricity production, wind, >3MW turbine, onshore',
   'code': '0d48975a3766c13e68cedeb6c24f6f74'},
  'output': ['kilowatt_hour', 3]},
 {'id': {'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'code': 'ed3da88fc23311ee183e9ffd376de89b'}}]

In [7]:
# select 2 random methods and convert them into the form for enbios2
methods = [bw2data.methods.random() for _ in range(2)]
experiment_methods = [{"id": method} for method in methods]

experiment_methods

[{'id': ('ReCiPe 2016 v1.03, midpoint (H)',
   'ozone depletion',
   'ozone depletion potential (ODPinfinite)')},
 {'id': ('ReCiPe 2016 v1.03, midpoint (H)',
   'land use',
   'agricultural land occupation (LOP)')}]

In [ ]:
# let's store the raw data, because we want to modify it later
simple_raw_data = {
    "bw_project": PROJECT_NAME,
    "activities": experiment_activities,
    "methods": experiment_methods,
}

# make a first validation of the experiment data
exp_data = ExperimentData(**simple_raw_data)

In [ ]:
# create experiment object. This will validate the activities, their outputs, the methods and the scenarios.
simple_experiment: Experiment = Experiment(exp_data)

In [ ]:
print(simple_experiment.info())

## Running the experiment

In [ ]:
# run all scenarios at once
results = simple_experiment.run()

# Result
The result is a dictionary of scenario names, where for each scenario we have a tree (representing the activity hierarchy). Each node (`BasicTreeNode`) in the tree has a `data` object, which is of the type `ScenarioResultNodeData`, which have the fields `output`, `result` and `bw_activity`.

In [ ]:
# from enbios.generic.tree.basic_tree import BasicTreeNode
# from enbios.models.experiment_models import ScenarioResultNodeData

results

In [ ]:
print(results["default scenario"].info())
print("---")
for children in results["default scenario"]:
    print(children.info())
    print("---")

In [ ]:
# we can dump the results into a csv file
simple_experiment.results_to_csv("test.csv")
pd.read_csv("test.csv").fillna("")

In [ ]:
simple_experiment.scenarios[0].result_to_dict()

## Add a technology hierarchy (dendrogram) 
Let's now add a few more activities to the experiment and create a hierarchy of activities.

In [ ]:
solar_spain = db.search("solar", filter={"location": "ES"})[:2]
solar_spain

In [ ]:
hierarchy_raw_data = deepcopy(simple_raw_data)

hierarchy_raw_data["activities"].extend(
    [
        {"id": {"name": activity["name"], "code": activity["code"]}}
        for activity in solar_spain
    ]
)

In [ ]:
hierarchy = {
    "wind": [wind_act["name"] for wind_act in wind_turbines_spain],
    "solar": [solar_act["name"] for solar_act in solar_spain],
}

hierarchy_raw_data["hierarchy"] = hierarchy
hierarchy

In [ ]:
hierarchy_experiment = Experiment(hierarchy_raw_data)
hierarchy_experiment

# Run the 2nd experiment

In [ ]:
hierarchy_experiment.run()

In [ ]:
# print(json.dumps((exp.scenarios[0].result_to_dict()), indent=2))
hierarchy_experiment.scenarios[0].results_to_csv(
    "test.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("test.csv").fillna("")

## Create several scenarios

In [ ]:
from random import randint

activity_aliases = hierarchy_experiment.activities_aliases


def create_random_scenario():
    return {
        "activities": {act: ["kilowatt_hour", randint(1, 10)] for act in activity_aliases}
    }


scenarios_raw_data = deepcopy(hierarchy_raw_data)

scenarios_raw_data["scenarios"] = [create_random_scenario(), create_random_scenario()]

scenarios_raw_data["scenarios"]

In [ ]:
scenarios_experiment = Experiment(scenarios_raw_data)

## Run the experiment for the 3rd time
This time will likely take some more time since we need to run 2 scenarios. 

In [ ]:
_ = scenarios_experiment.run()

In [ ]:
scenarios_experiment.scenarios[0].results_to_csv(
    "s1.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("s1.csv").fillna("")

## Inspecting the results

We can now do some transformations of the results. For that is useful to know how to retrieve is singular result from a scenario result. 
The result of a scenario is a tree structure, where the nodes `name`s are activity aliases or names defined in the hierarchy. With the function of BasicTreeNode.find_child_by_name we can directly access the result of a node.    

Following we transform the results into a dictionary of the following structure:
```json
{
    "activity_alias": {
        "method_alias": "[list of results for each scenario]"
    }
}
```

In [ ]:
all_results = {}
for activity in activity_aliases:
    all_results[activity] = {
        method_alias: [] for method_alias in scenarios_experiment.method_aliases
    }
    for scenario in scenarios_experiment.scenarios:
        activity_result = scenario.result_tree.find_child_by_name(activity)
        for method, score in activity_result.data.results.items():
            all_results[activity][method].append(score)

all_results

In [ ]:
distribution_raw_data = deepcopy(simple_raw_data)
distribution_raw_data["config"] = {"use_k_bw_distributions": 5}

In [ ]:
distribution_experiment = Experiment(distribution_raw_data)
results = distribution_experiment.run()

In [ ]:
results["default scenario"].data.distribution_results